## ebs.ErrorBudget.run_etc Required Arguments
### Fixed:  
* output_dir  
* ref_json_filename  
* pp_json_filename  
* contrast_filename  
* target_list  
* luminosity  
* eeid  
* eepsr  
* npoints 

### Non-variable, mutable  
### Possibly variable  
* exo_zodi  
* wfe
* wfsc_factor
* sensitivity
* output_filename_prefix
* var_par
* *args  

In [1]:
import yaml
import numpy as np
import utilities.log_pdf as pdf

In [2]:
pdf.uniform

<function utilities.log_pdf.uniform(x, lower_bound, upper_bound)>

In [3]:
with open('mcmc.yml', 'r') as config:
    config = yaml.load(config, Loader=yaml.FullLoader)

In [4]:
config

{'paths': {'output': './output', 'input': './inputs'},
 'input_files': {'contrast': 'mcmc_contrast.csv',
  'throughput': 'mcmc_throughput.csv'},
 'json_files': {'ref_json': 'mcmc_ref.json', 'pp_json': 'mcmc_pp.json'},
 'targets': {'star1': {'HIP': 32439,
   'spec_type': 'F8V',
   'luminosity': 0.2615,
   'eeid': 0.07423,
   'eepsr': 6.34e-11,
   'exo_zodi': 3.0},
  'star2': {'HIP': 77052,
   'spec_type': 'G5V',
   'luminosity': -0.0788,
   'eeid': 0.06174,
   'eepsr': 1.39e-10,
   'exo_zodi': 3.0},
  'star3': {'HIP': 79672,
   'spec_type': 'G2Va',
   'luminosity': 0.0391,
   'eeid': 0.07399,
   'eepsr': 1.06e-10,
   'exo_zodi': 3.0},
  'star4': {'HIP': 26779,
   'spec_type': 'K1V',
   'luminosity': -0.3209,
   'eeid': 0.05633,
   'eepsr': 2.42e-10,
   'exo_zodi': 3.0},
  'star5': {'HIP': 113283,
   'spec_type': 'K4Ve',
   'luminosity': -0.7,
   'eeid': 0.05829,
   'eepsr': 5.89e-10,
   'exo_zodi': 3.0}},
 'wfe': [[10.0, 10.0, 10.0, 10.0],
  [10.0, 10.0, 10.0, 10.0],
  [10.0, 10.0, 10.0

In [5]:
print(np.array(config['prior_args'][config['variable_id'][0]][0]))

[[9.9999e-07 9.9999e-07 9.9999e-07 9.9999e-07]
 [9.999e-05 9.999e-05 9.999e-05 9.999e-05]
 [0.00990099 0.00990099 0.00990099 0.00990099]
 [None None None None]
 [None None None None]]


In [6]:
print(np.array(config['prior_args'][config['variable_id'][0]][1]))

[[9.999e-05 9.999e-05 9.999e-05 9.999e-05]
 [0.00990099 0.00990099 0.00990099 0.00990099]
 [0.5 0.5 0.5 0.5]
 [None None None None]
 [None None None None]]


In [7]:
print("{:e}".format(1/(1+1.001e4)))

9.989012e-05
